In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import logging

# Configure logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

# -----------------------------------------------------------------
# 1. DEFINE THE STYLE
# -----------------------------------------------------------------
plt.rcParams['figure.figsize'] = [8, 6]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 14

# -----------------------------------------------------------------
# 2. DEFINE dlc COLORS
# -----------------------------------------------------------------
dlc = {
    "dlblue": "#0096ff",
    "dlorange": "#FF9300",
    "dldarkred": "#C00000",
    "dlmagenta": "#FF40FF",
    "dlpurple": "#7030A0",
    "dldarkblue": "#0D5BDC"
}

# -----------------------------------------------------------------
# 3. DEFINE COFFEE DATA FUNCTIONS
# -----------------------------------------------------------------
def load_coffee_data():
    """Creates coffee roasting data for classification."""
    X = np.array([[200, 13.9],  # Good roast
                  [214, 17.5],
                  [220, 15.8],
                  [205, 15.4],
                  [188, 16.6],  # Bad roast
                  [175, 19.2],
                  [185, 17.9],
                  [168, 18.1]])
    y = np.array([1, 1, 1, 1, 0, 0, 0, 0])
    return X, y

def plt_roast(X, y):
    """Plot coffee roasting data with good/bad roast classification"""
    Y = y.reshape(-1,)
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    ax.scatter(X[Y==0, 0], X[Y==0, 1], s=100, marker='x', c='red', label="Bad Roast")
    ax.scatter(X[Y==1, 0], X[Y==1, 1], s=100, marker='o', c='blue', label="Good Roast")
    ax.set_xlabel("Temperature \N{DEGREE SIGN}C")
    ax.set_ylabel("Duration (min)")
    ax.set_title("Coffee Roasting Data")
    ax.legend()
    ax.grid(True)
    plt.show()

def plt_prob(model, X, y):
    """Plot probability predictions"""
    yhat = model.predict(X)
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    sort_indices = np.argsort(X[:, 0])
    X_sorted = X[sort_indices]
    yhat_sorted = yhat[sort_indices]
    ax.plot(X_sorted[:, 0], yhat_sorted, c=dlc["dlblue"], linewidth=2, label="Prediction")
    ax.scatter(X[y==0, 0], y[y==0], s=100, marker='x', c='red', label="Bad Roast")
    ax.scatter(X[y==1, 0], y[y==1], s=100, marker='o', c='blue', label="Good Roast")
    ax.set_xlabel("Temperature \N{DEGREE SIGN}C")
    ax.set_ylabel("Probability of Good Roast")
    ax.set_title("Model Predictions")
    ax.legend()
    ax.grid(True)
    plt.show()

def plt_layer(X, Y, predicted, layer_name):
    """Plot layer activations"""
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    h = 0.1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = predicted(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, alpha=0.3, cmap='RdYlBu')
    ax.scatter(X[Y==0, 0], X[Y==0, 1], s=100, marker='x', c='red', label="Bad Roast")
    ax.scatter(X[Y==1, 0], X[Y==1, 1], s=100, marker='o', c='blue', label="Good Roast")
    ax.set_xlabel("Temperature \N{DEGREE SIGN}C")
    ax.set_ylabel("Duration (min)")
    ax.set_title(f"Layer: {layer_name}")
    ax.legend()
    ax.grid(True)
    plt.show()

def plt_network(X, y, model):
    """Visualize neural network predictions"""
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    h = 0.1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = (Z > 0.5).astype(int).reshape(xx.shape)
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    ax.contourf(xx, yy, Z, alpha=0.3, cmap='RdYlBu')
    ax.scatter(X[y==0, 0], X[y==0, 1], s=100, marker='x', c='red', label="Bad Roast")
    ax.scatter(X[y==1, 0], X[y==1, 1], s=100, marker='o', c='blue', label="Good Roast")
    ax.set_xlabel("Temperature \N{DEGREE SIGN}C")
    ax.set_ylabel("Duration (min)")
    ax.set_title("Neural Network Decision Boundary")
    ax.legend()
    ax.grid(True)
    plt.show()

def plt_output_unit(weights, bias):
    """Visualize single neuron parameters"""
    print(f"Weights: {weights.flatten()}")
    print(f"Bias: {bias[0]}")
    fig, ax = plt.subplots(1, 1, figsize=(6, 4))
    x_pos = np.arange(len(weights))
    ax.bar(x_pos, weights.flatten(), color=dlc["dlblue"])
    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax.set_xlabel("Input Feature")
    ax.set_ylabel("Weight Value")
    ax.set_title(f"Neuron Weights (Bias = {bias[0]:.2f})")
    ax.set_xticks(x_pos)
    ax.set_xticklabels(['Neuron 1', 'Neuron 2', 'Neuron 3'])
    ax.grid(True, alpha=0.3)
    plt.show()

# -----------------------------------------------------------------
# 4. MAIN EXECUTION CODE
# -----------------------------------------------------------------

# Load and visualize data
X, y = load_coffee_data()
print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"X samples:\n{X}")
print(f"y labels: {y}")
plt_roast(X, y)

# Normalize data
print(f"\nTemperature Max, Min pre normalization: {np.max(X[:,0]):0.2f}, {np.min(X[:,0]):0.2f}")
print(f"Duration    Max, Min pre normalization: {np.max(X[:,1]):0.2f}, {np.min(X[:,1]):0.2f}")
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X)  # learns mean, variance
Xn = norm_l(X)
print(f"\nTemperature Max, Min post normalization: {np.max(Xn[:,0]):0.2f}, {np.min(Xn[:,0]):0.2f}")
print(f"Duration    Max, Min post normalization: {np.max(Xn[:,1]):0.2f}, {np.min(Xn[:,1]):0.2f}")

# Create duplicated data (for training)
Xt = np.tile(Xn, (1000, 1))  # 1000 copies vertically
Yt = np.tile(y.reshape(-1, 1), (1000, 1))   
print(f"\nXt shape: {Xt.shape}, Yt shape: {Yt.shape}")

# Build neural network
tf.random.set_seed(1234)  # for consistent results
model = Sequential([
    tf.keras.Input(shape=(2,)),
    Dense(3, activation='sigmoid', name='layer1'),
    Dense(1, activation='sigmoid', name='layer2')
])

# Calculate parameters
L1_num_params = 2 * 3 + 3   # W1 parameters + b1 parameters
L2_num_params = 3 * 1 + 1   # W2 parameters + b2 parameters
print(f"\nL1 params = {L1_num_params}, L2 params = {L2_num_params}")
print(f"Total parameters: {L1_num_params + L2_num_params}")

# Train the model
print("\n--- TRAINING THE MODEL ---")
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
)
model.fit(Xt, Yt, epochs=10)

# Extract and display weights
print("\n--- MODEL WEIGHTS ---")
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print(f"W1{W1.shape}:\n", W1, f"\nb1{b1.shape}:", b1)
print(f"\nW2{W2.shape}:\n", W2, f"\nb2{b2.shape}:", b2)

# Make predictions
print("\n--- MAKING PREDICTIONS ---")
X_test = np.array([[200, 13.9],   # Should predict 1 (good)
                   [175, 19.2]])  # Should predict 0 (bad)
X_test_norm = norm_l(X_test)
predictions = model.predict(X_test_norm)
print(f"Raw predictions (probabilities):\n{predictions}")

# Convert to binary decisions
yhat = np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        yhat[i] = 1
    else:
        yhat[i] = 0
print(f"Binary decisions:\n{yhat}")

# Visualizations
print("\n--- VISUALIZATIONS ---")
plt_prob(model, Xn, y)

# Set pre-trained weights (for comparison/consistency)
print("\n--- SETTING PRE-TRAINED WEIGHTS ---")
W1_trained = np.array([[-8.94,  0.29, 12.89],
                       [-0.17, -7.34, 10.79]])
b1_trained = np.array([-9.87, -9.28,  1.01])
W2_trained = np.array([[-31.38],
                       [-27.86],
                       [-32.79]])
b2_trained = np.array([15.54])

model.get_layer("layer1").set_weights([W1_trained, b1_trained])
model.get_layer("layer2").set_weights([W2_trained, b2_trained])

# Show layer visualizations
print("\n--- LAYER VISUALIZATIONS ---")
plt_layer(X, y.reshape(-1,), lambda x: model.get_layer("layer1")(norm_l(x)), "layer1")
plt_output_unit(W2_trained, b2_trained)

# Show final network decision boundary
print("\n--- FINAL DECISION BOUNDARY ---")
netf = lambda x: model.predict(norm_l(x))
plt_network(X, y, netf)

print("\n✅ Program completed successfully!")